# 8. Complexity measures and correlation with computation times

In this notebook we compute 30 different molecular complexities or descriptors and their Pearson correlation with the generation and enumeration computation times.

In [1]:
import gzip
import json
import os
import sys

import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from collections import Counter, defaultdict
from rdkit import Chem, RDLogger
from rdkit.Chem import AllChem, Descriptors, RDConfig, rdDistGeom, rdMolDescriptors
from rdkit.Chem.rdmolops import GetDistanceMatrix
from rdkit.Chem.SpacialScore import SPS
from scipy.stats import pearsonr

# Add the folder containing ChiralDescriptors.py to the Python path
chiral_path = os.path.join(RDConfig.RDContribDir, 'ChiralPairs')
sys.path.append(chiral_path)
sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))
import sascorer

# Now import it
import ChiralDescriptors

from molsig.Signature import MoleculeSignature
from collections import Counter

# Results path

In [3]:
path_results = "C:/Users/meyerp/Documents/INRAE/Diophantine/Enumération/github/results/"

# Database choice

In [2]:
database_chosen = "metanetx"
#database_chosen = "emolecules"

# Compute and export molecular complexities

## Molecular complexity functions

- ecfp complexities

In [3]:
fpgen = AllChem.GetMorganGenerator(radius=2, fpSize=2048, includeChirality=True)

def get_ecfp_bits_sum(smi):
    mol = Chem.MolFromSmiles(smi)
    morgan = fpgen.GetCountFingerprint(mol).ToList()
    return sum(morgan)

def get_ecfp_bits_max(smi):
    mol = Chem.MolFromSmiles(smi)
    morgan = fpgen.GetCountFingerprint(mol).ToList()
    return max(morgan)

def get_ecfp_bits_prod(smi):
    mol = Chem.MolFromSmiles(smi)
    morgan = fpgen.GetCountFingerprint(mol).ToList()
    return np.log(np.prod([m for m in morgan if m > 0]))

- molcular signature complexities

In [4]:
def count_string_occurrences(lst):
    """
    Convert a list of strings into a vector of counts of unique string occurrences.

    Parameters:
    lst (list of str): Input list of strings

    Returns:
    list of int: Counts of unique strings (unordered)
    """
    counter = Counter(lst)
    return list(counter.values())

def smi_to_sig_vector(smi):
    mol = Chem.MolFromSmiles(smi)
    # Compute molecular signature
    ms = MoleculeSignature(mol, radius=2, nbits=0, map_root=True, use_stereo=False)
    ms.post_compute_neighbors()
    sig = sorted([atom.to_string(neighbors=True) for atom in ms.atoms])
    sig_vector = count_string_occurrences(sig)
    return sig_vector

In [5]:
def get_sig_bits_sum(smi):
    sig_vector = smi_to_sig_vector(smi)
    return sum(sig_vector)

def get_sig_bits_max(smi):
    sig_vector = smi_to_sig_vector(smi)
    return max(sig_vector)

def get_sig_bits_prod(smi):
    sig_vector = smi_to_sig_vector(smi)
    return np.log(np.prod([m for m in sig_vector if m > 0]))

###  From *Buehler, Y., & Reymond, J. L. (2025). A View on Molecular Complexity from the GDB Chemical Space. Journal of Chemical Information and Modeling.*

In [6]:
# The sum of on-bits in an ECFP4 as a complexity measure
def calculate_fcfp4_complexity(smiles):
    # Convert SMILES to RDKit molecule object
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print(f"Invalid SMILES: {smiles}")
        return None
    
    
    # Suppress RDKit warnings
    RDLogger.DisableLog('rdApp.*')
    
    # Generate FCFP_4 fingerprint (functional-based ECFP4)
    # Use the 'useFeatures=True' argument to focus on functional groups
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048, useFeatures=True)

    # Calculate the complexity score by counting on-bits
    complexity_score = fp.GetNumOnBits()

    return complexity_score

In [7]:
# data warrior
# You can use the [**DataWarrior**](https://openmolecules.org/datawarrior/download.html) software to calculate the complexity

In [8]:
# Code adapted from: https://github.com/boskovicgroup/bottchercomplexity
# Current failures: Does not distinguish between cyclopentyl and pentyl (etc.)
#                   and so unfairly underestimates complexity.

def GetChemicalNonequivs(atom, themol):
    num_unique_substituents = 0
    substituents = [[],[],[],[]]
    for item,key in enumerate(ChiralDescriptors.determineAtomSubstituents(atom.GetIdx(), themol, Chem.GetDistanceMatrix(themol))[0]):
        for subatom in ChiralDescriptors.determineAtomSubstituents(atom.GetIdx(), themol, Chem.GetDistanceMatrix(themol))[0][key]:
            substituents[item].append(themol.GetAtomWithIdx(subatom).GetSymbol())
            num_unique_substituents = len(set(tuple(tuple(substituent) for substituent in substituents if substituent)))
            #
            # Logic to determine e.g. whether repeats of CCCCC are cyclopentyl and pentyl or two of either
            #
    return num_unique_substituents

#
# The number of different non-hydrogen elements or isotopes (including deuterium
# and tritium) in the atom's microenvironment.
#
# CH4 - the carbon has e_i of 1
# Carbonyl carbon of an amide e.g. CC(=O)N e_i = 3
#     while N and O have e_i = 2
#
def GetBottcherLocalDiversity(atom):
    neighbors = []
    for neighbor in atom.GetNeighbors():
        neighbors.append(str(neighbor.GetSymbol()))
    if atom.GetSymbol() in set(neighbors):
        return len(set(neighbors))
    else:
        return len(set(neighbors))+1

#
# RDKit marks atoms where there is potential for isomerization with a tag 
# called _CIPCode. If it exists for an atom, note that S = 2, otherwise 1. 
def GetNumIsomericPossibilities(atom):
    try:
        if(atom.GetProp('_CIPCode')):
            return 2
    except:
        return 1

#
# The number of valence electrons the atom would have if it were unbonded and
# neutral
# TODO: Move this dictionary somewhere else. 
def GetNumValenceElectrons(atom):
    valence = {1: ['H', 'Li', 'Na', 'K', 'Rb', 'Cs', 'Fr'], # Alkali Metals
               2: ['Be', 'Mg', 'Ca', 'Sr', 'Ba', 'Ra'], # Alkali Earth Metals
               #transition metals???
               3: ['B', 'Al', 'Ga', 'In', 'Tl', 'Nh'], #
               4: ['C', 'Si', 'Ge', 'Sn', 'Pb', 'Fl'],
               5: ['N', 'P', 'As', 'Sb', 'Bi', 'Mc'], # Pnictogens
               6: ['O', 'S', 'Se', 'Te', 'Po', 'Lv'], # Chalcogens
               7: ['F', 'Cl', 'Br', 'I', 'At', 'Ts'], # Halogens
               8: ['He', 'Ne', 'Ar', 'Kr', 'Xe', 'Rn', 'Og']} # Noble Gases
    for k in valence:
        if atom.GetSymbol() in valence[k]:
            return k
    return 0

#
# Represents the total number of bonds to other atoms with V_i*b_i > 1, so
# basically bonds to atoms other than Hydrogen
#
# Here we can leverage the fact that RDKit does not even report Hydrogens by
# default to simply loop over the bonds. We will have to account for molecules
# that have hydrogens turned on before we can submit this code as a patch
# though.
#
# TODO: Create a dictionary for atom-B value pairs for use when AROMATIC is detected in bonds.
def GetBottcherBondIndex(atom):
    b_sub_i_ranking = 0
    bonds = []
    for bond in atom.GetBonds():
        bonds.append(str(bond.GetBondType()))
    for bond in bonds:
        if bond == 'SINGLE':
            b_sub_i_ranking += 1
        if bond == 'DOUBLE':
            b_sub_i_ranking += 2
        if bond == 'TRIPLE':
            b_sub_i_ranking += 3
    if 'AROMATIC' in bonds:
        # This list can be expanded as errors arise.
        if atom.GetSymbol() == 'C':
            b_sub_i_ranking += 3
        elif atom.GetSymbol() == 'N':
            b_sub_i_ranking += 2
    return b_sub_i_ranking

def GetBottcherComplexity(smiles, debug=False):
    themol=Chem.MolFromSmiles(smiles)
    complexity = 0
    Chem.AssignStereochemistry(themol,cleanIt=True,force=True,flagPossibleStereoCenters=True)
    atoms = themol.GetAtoms();
    atom_stereo_classes = []
    atoms_corrected_for_symmetry = []
    for atom in atoms:
        if atom.GetProp('_CIPRank') in atom_stereo_classes:
            continue
        else:
            atoms_corrected_for_symmetry.append(atom)
            atom_stereo_classes.append(atom.GetProp('_CIPRank'))
    for atom in atoms_corrected_for_symmetry:
        d = GetChemicalNonequivs(atom, themol)
        e = GetBottcherLocalDiversity(atom)
        s = GetNumIsomericPossibilities(atom)
        V = GetNumValenceElectrons(atom)
        b = GetBottcherBondIndex(atom)
        if V * b == 0:
            return None
        complexity += d*e*s*math.log(V*b,2)
        if debug:
            print(str(atom.GetSymbol()))
            print('\tSymmetry Class: ' + str(atom.GetProp('_CIPRank')))
            print('\tNeighbors: ')
            print('\tBonds: ')
            print('\tCurrent Parameter Values:')
            print('\t\td_sub_i: ' + str(d))
            print('\t\te_sub_i: ' + str(e))
            print('\t\ts_sub_i: ' + str(s))
            print('\t\tV_sub_i: ' + str(V))
            print('\t\tb_sub_i: ' + str(b))
    if debug:
        print('Current Complexity Score: ' + str(complexity))
        return
    return complexity

In [9]:
def calculate_path_complexity(smiles):
    # Convert SMILES to RDKit molecule object
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        print(f"Invalid SMILES: {smiles}")
        return None

    # Generate atom paths and path frequencies using Morgan fingerprint
    radius = 2  # Path length (corresponding to ECFP4)
    fingerprint = rdMolDescriptors.GetMorganFingerprint(mol, radius)
    paths = fingerprint.GetNonzeroElements()

    # Calculate path frequencies per atom environment
    atom_paths = defaultdict(list)
    for path, count in paths.items():
        atoms_in_path = path % mol.GetNumAtoms()  # Map path back to atom
        atom_paths[atoms_in_path].append(count)

    # Step 1: Calculate atomic complexity (CA)
    ca_values = {}
    for atom, path_counts in atom_paths.items():
        total_paths = sum(path_counts)
        path_fractions = [count / total_paths for count in path_counts]
        ca = -sum(p * math.log2(p) for p in path_fractions) + math.log2(total_paths)
        ca_values[atom] = ca

    # Step 2: Calculate molecular complexity (CM)
    cm = sum(ca_values.values())

    # Step 3: Calculate log-sum complexity (CM*)
    # cm_star = math.log2(sum(2 ** ca for ca in ca_values.values()))

    # Step 4: Calculate structural entropy complexity (CSE)
    # atom_types = [atom.GetAtomicNum() for atom in mol.GetAtoms()]
    # total_atoms = len(atom_types)
    # type_frequencies = {atype: atom_types.count(atype) / total_atoms for atype in set(atom_types)}
    # cse = -sum(freq * math.log2(freq) for freq in type_frequencies.values())

    return cm
        #"CA (per atom)": ca_values,
        #"CM (molecular complexity)": cm
        #"CM* (log-sum complexity)": cm_star,
        #"CSE (structural entropy complexity)": cse

In [10]:
# SPS calculation
def sps(smiles):
    mol = Chem.MolFromSmiles(smiles)
    sps = SPS(mol, normalize=False)
    return sps

In [11]:
# nSPS calculation
def nsps(smiles):
    mol = Chem.MolFromSmiles(smiles)
    nsps = SPS(mol, normalize=True)
    return nsps

In [12]:
# Calculate the SAS
def sascore(smiles):
    mol=Chem.MolFromSmiles(smiles)
    sas_score = sascorer.calculateScore(mol)
    return sas_score

- SCS

In [13]:
'''
This is a standalone, importable SCScorer model. It does not have tensorflow as a
dependency and is a more attractive option for deployment. The calculations are
fast enough that there is no real reason to use GPUs (via tf) instead of CPUs (via np)
'''

# Set the root directory where SCScorer is located.
# Download the source code from: https://github.com/connorcoley/scscore/tree/master
project_root = 'C:/Users/meyerp/Documents/INRAE/Diophantine/Enumération/scscore-master'  # <-- Update this path to match your local setup

score_scale = 5.0
min_separation = 0.25

FP_len = 1024
FP_rad = 2

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

class SCScorer():
    def __init__(self, score_scale=score_scale):
        self.vars = []
        self.score_scale = score_scale
        self._restored = False

    def restore(self, weight_path=os.path.join(project_root, 'models', 'full_reaxys_model_1024bool', 'model.ckpt-10654.as_numpy.pickle'), FP_rad=FP_rad, FP_len=FP_len):
        self.FP_len = FP_len; self.FP_rad = FP_rad
        self._load_vars(weight_path)
        #print('Restored variables from {}'.format(weight_path))

        if 'uint8' in weight_path or 'counts' in weight_path:
            def mol_to_fp(self, mol):
                if mol is None:
                    return np.array((self.FP_len,), dtype=np.uint8)
                fp = AllChem.GetMorganFingerprint(mol, self.FP_rad, useChirality=True) # uitnsparsevect
                fp_folded = np.zeros((self.FP_len,), dtype=np.uint8)
                for k, v in six.iteritems(fp.GetNonzeroElements()):
                    fp_folded[k % self.FP_len] += v
                return np.array(fp_folded)
        else:
            def mol_to_fp(self, mol):
                if mol is None:
                    return np.zeros((self.FP_len,), dtype=np.float32)
                return np.array(AllChem.GetMorganFingerprintAsBitVect(mol, self.FP_rad, nBits=self.FP_len,
                    useChirality=True), dtype=np.bool_)
        self.mol_to_fp = mol_to_fp

        self._restored = True
        return self

    def smi_to_fp(self, smi):
        if not smi:
            return np.zeros((self.FP_len,), dtype=np.float32)
        return self.mol_to_fp(self, Chem.MolFromSmiles(smi))

    def apply(self, x):
        if not self._restored:
            raise ValueError('Must restore model weights!')
        # Each pair of vars is a weight and bias term
        for i in range(0, len(self.vars), 2):
            last_layer = (i == len(self.vars)-2)
            W = self.vars[i]
            b = self.vars[i+1]
            x = np.matmul(x, W) + b
            if not last_layer:
                x = x * (x > 0) # ReLU
        x = 1 + (score_scale - 1) / (1 + np.exp(-x))
        return x

    def get_score_from_smi(self, smi='', v=False):
        if not smi:
            return ('', 0.)
        fp = np.array((self.smi_to_fp(smi)), dtype=np.float32)
        if sum(fp) == 0:
            if v: print('Could not get fingerprint?')
            cur_score = 0.
        else:
            # Run
            cur_score = self.apply(fp)
            if v: print('Score: {}'.format(cur_score))
        mol = Chem.MolFromSmiles(smi)
        if mol:
            smi = Chem.MolToSmiles(mol, isomericSmiles=True, kekuleSmiles=True)
        else:
            smi = ''
        return (smi, cur_score)

    def _load_vars(self, weight_path):
        if weight_path.endswith('pickle'):
            import cPickle as pickle
            with open(weight_path, 'rb') as fid:
                self.vars = pickle.load(fid)
                self.vars = [x.tolist() for x in self.vars]
        elif weight_path.endswith('json.gz'):
            with gzip.GzipFile(weight_path, 'r') as fin:    
                json_bytes = fin.read()                      
                json_str = json_bytes.decode('utf-8')            
                self.vars = json.loads(json_str)
                self.vars = [np.array(x) for x in self.vars]

In [14]:
# Define your SMILES string
smi = "COC1=C(O)C=C(CC(=O)O)C=C1Br"

# Create a DataFrame
df = pd.DataFrame({'SMILES': [smi]})

df

,SMILES
0,COC1=C(O)C=C(CC(=O)O)C=C1Br


In [15]:
if __name__ == '__main__':

    # Ensure the input file has at least one column
    if 'SMILES' not in df.columns:
        raise ValueError("Input file must have a 'smiles' column")

    # Initialize the model
    model = SCScorer()
    model.restore(os.path.join(project_root, 'models', 'full_reaxys_model_1024bool', 'model.ckpt-10654.as_numpy.json.gz'))

    # Process each SMILES string and store the scores
    scores = []
    for smi in df['SMILES']:
        _, sco = model.get_score_from_smi(smi)
        scores.append(float(sco[0])) 


    # Add a new column to the DataFrame for the scores
    df['SCS'] = scores
df

[14:15:51] DEPRECATION WARNING: please use MorganGenerator


,SMILES,SCS
0,COC1=C(O)C=C(CC(=O)O)C=C1Br,2.334439


In [16]:
# Load SCS model once
model = SCScorer()
model_path = os.path.join(project_root, 'models', 'full_reaxys_model_1024bool', 'model.ckpt-10654.as_numpy.json.gz')
model.restore(model_path)

def get_scs_score(smi):
    try:
        _, score = model.get_score_from_smi(smi)
        return float(score[0])
    except Exception as e:
        print(f"SCS error for {smi}: {e}")
        return None

- MCs

In [17]:
# Calculate the FDV
def divalent_nodes_fraction(smiles):
    mol=Chem.MolFromSmiles(smiles)
    atom_number = 0
    divalent_node = 0
    
    for atom in mol.GetAtoms():
        atom_number += 1
        degree = atom.GetDegree()

        if degree == 2:
            divalent_node += 1 
        else:
            continue

    divalent_ratio = round(divalent_node/atom_number,2)
    return divalent_ratio

#: MC1 calculation
def mc1(smiles):

    mc1 = 1- divalent_nodes_fraction(smiles)
    
    return mc1

In [18]:
# MC2 calculation - Count non-divalent atoms that are not in the C=O-X double bonds 
def mc2(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        raise ValueError("Invalid SMILES string.")

    atoms_in_C_O_X_double_bond = set()

    for bond in mol.GetBonds():
        # Check for a C=O double bond
        if bond.GetBondType() == Chem.rdchem.BondType.DOUBLE:
            begin_atom = bond.GetBeginAtom()
            end_atom = bond.GetEndAtom()

            if {begin_atom.GetAtomicNum(), end_atom.GetAtomicNum()} == {6, 8}:  # C and O
                # Identify which one is the carbon
                carbon = begin_atom if begin_atom.GetAtomicNum() == 6 else end_atom
                oxygen = end_atom if carbon == begin_atom else begin_atom

                # Check carbon's neighbors for N or O (excluding the double-bonded O)
                for neighbor in carbon.GetNeighbors():
                    if neighbor.GetIdx() != oxygen.GetIdx() and neighbor.GetAtomicNum() in [7, 8]:
                        atoms_in_C_O_X_double_bond.update([carbon.GetIdx(), oxygen.GetIdx()])
                        break  # Only need one N/O neighbor to satisfy the condition

    # Count non-divalent atoms not in C=O-X double bonds
    count = 0
    for atom in mol.GetAtoms():
        if atom.GetDegree() != 2 and atom.GetIdx() not in atoms_in_C_O_X_double_bond:
            count += 1

    return count

### Others measures

In [19]:
def get_aromatic_proportion(smi):
    mol = Chem.MolFromSmiles(smi)
    if mol is None:
        return None
    aromatic_atoms = sum(1 for atom in mol.GetAtoms() if atom.GetIsAromatic())
    total_atoms = mol.GetNumAtoms()
    return aromatic_atoms / total_atoms if total_atoms > 0 else 0

In [20]:
def wiener_index(smi):
    mol = Chem.MolFromSmiles(smi)
    dmat = GetDistanceMatrix(mol)
    # sum only upper triangle, no double count
    return dmat[np.triu_indices_from(dmat, k=1)].sum()

In [21]:
def shannon_entropy_atom_types(smi):
    mol = Chem.MolFromSmiles(smi)
    atom_types = [atom.GetSymbol() for atom in mol.GetAtoms()]
    counts = Counter(atom_types)
    total = sum(counts.values())
    
    entropy = -sum((count / total) * math.log2(count / total) for count in counts.values())
    return entropy

In [22]:
def harary_index(smi):
    mol = Chem.MolFromSmiles(smi)
    dmat = GetDistanceMatrix(mol)
    dmat = dmat[np.triu_indices_from(dmat, k=1)]
    # Avoid division by zero
    dmat = dmat[dmat > 0]
    return np.sum(1.0 / dmat)

# All molecular complexities

In [24]:
# all
complexity_functions = {
    # descriptors
    "Nb atoms": lambda smi: (Chem.MolFromSmiles(smi)).GetNumAtoms(),
    "Nb bonds": lambda smi: (Chem.MolFromSmiles(smi)).GetNumBonds(),
    "Molecular weight": lambda smi: Descriptors.MolWt(Chem.MolFromSmiles(smi)),
    "ECFP4 sum": get_ecfp_bits_sum,
    "ECFP4 max": get_ecfp_bits_max,
    "ECFP4 prod": get_ecfp_bits_prod,
    "Sig max": get_sig_bits_max,
    "Sig prod": get_sig_bits_prod,
    "SMILES length": lambda smi: len(smi),
    "Wiener index": wiener_index,
    "Shannon entropy (atoms)": shannon_entropy_atom_types,
    "Harary index": harary_index,
    # gtp descriptors
    "LogP": lambda smi: Descriptors.MolLogP(Chem.MolFromSmiles(smi)),
    "TPSA": lambda smi: Descriptors.TPSA(Chem.MolFromSmiles(smi)),
    "H-bond donors": lambda smi: Descriptors.NumHDonors(Chem.MolFromSmiles(smi)),
    "H-bond acceptors": lambda smi: Descriptors.NumHAcceptors(Chem.MolFromSmiles(smi)),
    "Rotatable bonds": lambda smi: Descriptors.NumRotatableBonds(Chem.MolFromSmiles(smi)),
    "Ring count": lambda smi: Descriptors.RingCount(Chem.MolFromSmiles(smi)),
    "Fraction Csp3": lambda smi: Descriptors.FractionCSP3(Chem.MolFromSmiles(smi)),
    #"Aromatic proportion": get_aromatic_proportion,
    "Balaban index": lambda smi: Descriptors.BalabanJ(Chem.MolFromSmiles(smi)),
    "Bertz complexity": lambda smi: Descriptors.BertzCT(Chem.MolFromSmiles(smi)),
    # paper Buehler and Reymond
    "FCFP4 complexity": calculate_fcfp4_complexity,
    "Bottcher complexity": GetBottcherComplexity,
    "Proudfoot complexity": calculate_path_complexity,
    "SPS": sps,
    "nSPS": nsps,
    "sascore": sascore,
    "scscore": get_scs_score,
    "mc1": mc1,
    "mc2": mc2
}

len(complexity_functions)

30

## Molecular complexities computation

In [50]:
path_results_enum = path_results + "RevSig_" + database_chosen + ".xlsx"
df_results = pd.read_excel(path_results_enum, index_col=0)

# Initialize results list
results = []

# Loop over each SMILES
for smi in df_results["smi"]:
    row = {"SMILES": smi}
    for name, func in complexity_functions.items():
        try:
            row[name] = func(smi)
        except Exception as e:
            print(f"Error computing {name} for {smi}: {e}")
            row[name] = None
    results.append(row)

# Convert to DataFrame
df_complexities = pd.DataFrame(results)

# Preview or export
print(df_complexities.head())

df_complexities.to_excel("complexity_matrix_" + database_chosen + ".xlsx", index=False)

Error computing Bottcher complexity for N#Cc1ccc(S(F)(F)(F)(F)F)cc1: list index out of range
                                              SMILES  Nb atoms  Nb bonds  \
0      CN(C)CCN(C)C(=O)c1ccc(B2OC(C)(C)C(C)(C)O2)cn1        24        25   
1  NC(=O)c1c(-c2ccc(Cl)cc2Cl)csc1NC(=O)[C@@H]1CCC...        28        30   
2                      Oc1cc(F)ccc1CNCc1ccc(F)c(F)c1        19        20   
3  CC(C)(C)c1ccc(O)c(-n2nc(C(C)(C)C)c(N)c2C(F)(F)...        25        26   
4  CC(=O)c1ccc2c(c1)CCN2C(=O)C1CCN(C(=O)CCN2C(=O)...        35        39   

   Molecular weight  ECFP4 sum  ECFP4 max  ECFP4 prod  Sig max  Sig prod  \
0           333.241         64          7   13.154048        4  3.465736   
1           441.336         77          6   13.693045        2  1.386294   
2           267.250         53          6   10.227309        2  0.693147   
3           355.404         64          6   11.156022        6  2.890372   
4           473.529         99          7   21.391527        2  4.8520

# Correlations

- log

In [25]:
# Function to apply the style per column
def highlight_max_in_column(s):
    # skip first column → s will be a Series corresponding to one column of numbers
    is_max = s == s.max()
    return ['background-color: red' if v else '' for v in is_max]

def compute_complexity_correlation_matrix(df_complexities, df_results, ct_columns, output_excel=None):
    """
    Compute a DataFrame of Pearson correlations between each complexity descriptor
    and log(CT) values from multiple CT columns.

    Parameters:
    - df_complexities: DataFrame with 'SMILES' and complexity columns
    - df_results: DataFrame with 'smi' and CT columns
    - ct_columns: list of CT column names in df_results
    - output_excel: optional path to save the result as an Excel file

    Returns:
    - pd.DataFrame with columns: ['Complexity name', <CT1>, <CT2>, ...]
    """
    # Get all complexity descriptor names (excluding SMILES)
    complexity_names = [col for col in df_complexities.columns if col != "SMILES"]

    # Initialize result dictionary
    correlation_dict = {"Complexity name": complexity_names}

    for ct_column in ct_columns:
        correlations = []

        # Merge on SMILES to align descriptors with CTs
        df_merged = pd.merge(df_complexities, df_results[["smi", ct_column]],
                             left_on="SMILES", right_on="smi")

        raw_CT = df_merged[ct_column].tolist()

        for name in complexity_names:
            values = df_merged[name].tolist()

            # Filter valid entries
            filtered_data = [
                (v, ct) for v, ct in zip(values, raw_CT)
                if pd.notnull(v) and ct > 0
            ]

            if not filtered_data:
                correlations.append(np.nan)
            else:
                x_vals, ct_vals = zip(*filtered_data)
                log_ct_vals = np.log(ct_vals)
                corr, _ = pearsonr(x_vals, log_ct_vals)
                correlations.append(corr)

        correlation_dict[ct_column] = correlations

    # Create and optionally export DataFrame
    df_corr_matrix = pd.DataFrame(correlation_dict)
    
    if output_excel:
        #df_corr_matrix.to_excel(output_excel, index=False)
        styler = df_corr_matrix.style
        for col in df_corr_matrix.columns[1:]:
            styler = styler.apply(highlight_max_in_column, subset=[col])
        styler.to_excel(output_excel, index=False)
        print(f"Exported correlation matrix to {output_excel}")
    
    return df_corr_matrix

- log, raw, square, sqrt, inv_log, loglog

In [26]:
df_complexities_full = pd.ExcelFile(path_results + "RevSig_complexity_correlations.xlsx")

if database_chosen == "metanetx":
    df_complexities = pd.read_excel(df_complexities_full, "MetaNetX")
if database_chosen == "emolecules":
    df_complexities = pd.read_excel(df_complexities_full, "eMolecules")

print(df_complexities.shape)
print(df_complexities.head())

(10000, 31)
                                              SMILES  Nb atoms  Nb bonds  \
0                              CCN1C(=O)OC(C)(C)C1=O        11        11   
1     CC(=O)NC[C@H](O)c1c[nH]c(C(=O)[O-])c1C(=O)[O-]        18        18   
2      Nc1nc(N2CCC2)c2ncn([C@H]3C=C[C@@H](CO)C3)c2n1        21        24   
3  C[C@H](CO)C(=O)O[C@@H]1O[C@H](C(=O)O)[C@H](O)[...        19        19   
4  C[C@H](CCCCCCCCCCN(C)C)[C@H](CC(=O)[O-])C(=O)[O-]        23        22   

   Molecular weight  ECFP4 sum  ECFP4 max  ECFP4 prod  Sig max  Sig prod  \
0           157.169         28          3    3.871201        2  0.693147   
1           254.198         47          4    6.761573        2  1.386294   
2           286.339         60          4    9.534161        2  0.693147   
3           280.229         49          7    7.138867        3  1.098612   
4           327.465         62         11   12.219739        6  2.484907   

   SMILES length  ...  Bertz complexity  FCFP4 complexity  \
0            

## Method choice

In [29]:
for method_chosen in ["enum", "gen_top001", "gen_top010", "gen_top100"]:
    if method_chosen == "enum":
        path_results_enum = path_results + "RevSig_" + database_chosen + ".xlsx"
        df_results = pd.read_excel(path_results_enum, index_col=0)
        ct_columns = ["CT solve_partitions", "CT ecfp_sig", "CT sig_mol", "CT ecfp_mol"]
    else:
        path_results_gen = path_results + "generative/" + database_chosen + "." + method_chosen[-6:] + ".results.refined.tsv"
        df_results = pd.read_csv(path_results_gen, sep='\t')
        df_results.rename(columns={'Query SMILES': 'smi'}, inplace=True)
        ct_columns = ["Time Elapsed"]
    
    output_filename = "correlation_matrix_" + database_chosen + "_" + method_chosen + ".xlsx"
    df_corr_matrix = compute_complexity_correlation_matrix(df_complexities, df_results, ct_columns, output_excel=output_filename)
    print(df_corr_matrix)

Exported correlation matrix to correlation_matrix_metanetx_gen_top100.xlsx
            Complexity name  Time Elapsed
0                  Nb atoms      0.906591
1                  Nb bonds      0.901084
2          Molecular weight      0.888754
3                 ECFP4 sum      0.881220
4                 ECFP4 max      0.425821
5                ECFP4 prod      0.736042
6                   Sig max      0.114884
7                  Sig prod      0.287728
8             SMILES length      0.769275
9              Wiener index      0.742441
10  Shannon entropy (atoms)      0.045257
11             Harary index      0.889492
12                     LogP      0.151249
13                     TPSA      0.449872
14            H-bond donors      0.309171
15         H-bond acceptors      0.506446
16          Rotatable bonds      0.250396
17               Ring count      0.518250
18            Fraction Csp3      0.062893
19            Balaban index     -0.302040
20         Bertz complexity      0.772660
2